# 라이브러리

In [1]:
import os, shutil, zipfile
from tqdm import tqdm

import pandas as pd
import numpy as np

import re
import json

# 상수설정

In [2]:
data_path = "content"
# data_path = "./files"

STOPWORDS_CRAWLED = [["파스타","스파게티"]]
STOPWORDS_DB = [["만둣국","만두국"],["둘깨칼국수","들깨칼국수"]]

# 크롤링 된 급식 데이터 병합

## 분할 된 데이터 압출 풀기

In [3]:
with zipfile.ZipFile(os.path.join(data_path,"crawled_cafeteria_menu[0_301].zip"),"r") as target_file:
    target_file.extractall(data_path)
with zipfile.ZipFile(os.path.join(data_path,"crawled_cafeteria_menu[302_400].zip"),"r") as target_file:
    target_file.extractall(data_path)
with zipfile.ZipFile(os.path.join(data_path,"crawled_cafeteria_menu[401_899].zip"),"r") as target_file:
    target_file.extractall(data_path)
with zipfile.ZipFile(os.path.join(data_path,"crawled_cafeteria_menu[900_1800].zip"),"r") as target_file:
    target_file.extractall(data_path)
with zipfile.ZipFile(os.path.join(data_path,"crawled_cafeteria_menu[1801_9999].zip"),"r") as target_file:
    target_file.extractall(data_path)

## 데이터 병합

In [4]:
menu_csv_0 = pd.read_csv(os.path.join(data_path, "crawled_cafeteria[0_301].csv"),encoding="euc-kr")
menu_csv_302 = pd.read_csv(os.path.join(data_path, "crawled_cafeteria[302_400].csv"),encoding="euc-kr")
menu_csv_401 = pd.read_csv(os.path.join(data_path, "crawled_cafeteria[401_899].csv"),encoding="euc-kr")
menu_csv_900 = pd.read_csv(os.path.join(data_path, "crawled_cafeteria[900_1800].csv"),encoding="euc-kr")
menu_csv_1801 = pd.read_csv(os.path.join(data_path, "crawled_cafeteria[1801_9999].csv"),encoding="utf-8")

menu_csv = pd.concat([menu_csv_0, menu_csv_302, menu_csv_401, 
                      menu_csv_900, menu_csv_1801], axis=0)

menu_csv_0 = None ; menu_csv_302 = None ; menu_csv_401 = None 
menu_csv_900 = None ;menu_csv_1801 = None
menu_csv.tail()

menus = []
# 한글자 메뉴 제외하고 메뉴 리스트 생성
for menu in menu_csv.menu:
    matches = re.findall("[가-힣]{2,}", menu)
    menus.extend(matches)

# 중복 제거
menus = list(set(menus))
len(menus)

249238

### 추가 작업

#### 작업1
- A와B => [A, B]

In [5]:
extracted_split_menus = []

split_words = ["와","과","랑"]

exception_menus = [["크로와상","크로와쌍","크루와상","크루와쌍","크와상","크와쌍",
                   "와플","와인","하와이","하아와","와사비","와퍼","와시비","제노와즈",
                   "다쿠와즈","소와나무","와쿠와쿠","와구와구","요쿠와"],
                   ["사과","과일","과채","과즙","과즐","과립","견과","건과","과자","빙과","모과","약과","제과","과베기","과콰몰리",
                   "용과","무화과","백향과","하미과","한과","화과방","화과육"],
                   ["가다랑","후랑크","프랑크","동그랑땡","사랑","호랑","말랑","구프랑","명랑","나랑드","나드랑","노랑",
                   "네모랑땡","조랑떡","조랑이떡","그랑탕","몽블랑","베테랑","그랑땡","그랑떙","프랑스","랑그드샤","프랑쇼콜라"]]

for split_word_idx, split_word in enumerate(split_words):
    for menu_idx, menu in tqdm(enumerate(menus)):
        TF = False
        for exception_menu in exception_menus[split_word_idx]:
            if exception_menu in menu:
                TF = True
                break
        if TF: continue
        menu_list = []
        matches = re.findall(f"[가-힣]+{split_word}[가-힣]+", menu)
        if (split_word_idx>0):
            matches.extend(menu_list)
        for match_idx, match in enumerate(matches):
            split_list = match.split(split_word)
            for split_menu in split_list:
                if len(split_menu) < 2:continue
                menu_list.append(split_menu)
            menus[menu_idx] = ""
    extracted_split_menus.extend(menu_list)
    
menus.extend(extracted_split_menus)

menus = list(set(menus))

249238it [00:00, 337679.56it/s]
249238it [00:00, 325809.69it/s]
249238it [00:00, 312740.06it/s]


#### 작업2
- A와 => A

In [6]:
split_words = ["와","과","랑"]

exception_menus2 = [[],
                    ["사과","견과","건과","빙과","모과","약과","제과","유과",
                     "용과","무화과","백향과","하미과","한과","정과"],
                   ["구프랑","몽블랑"]]

for split_word_idx, split_word in enumerate(split_words):
    for menu_idx, menu in tqdm(enumerate(menus)):
        TF = False
        for exception_menu in exception_menus2[split_word_idx]:
            if exception_menu in menu:
                TF = True
                break
        if TF: continue
        menu_list = []
        matches = re.findall(f"[가-힣]+{split_word}$", menu)
        if(len(matches)>0):
            menus[menu_idx] = menu[:-1]
            if(len(menus)) == 0: menus[menu_idx] = ""

248118it [00:00, 633033.07it/s]
248118it [00:00, 412572.97it/s]
248118it [00:00, 581229.61it/s]


#### 작업3 불용어처리
- 포함되면 지워야 하는 목록
- 정확히 일치되야 지워야 하는 목록

In [7]:
rm_words = ["소스","쥬스","주스","아이스크림","케이크","케익","젤리",
            "아이스","아이스크림","쉐이크","요플레","요구르트","요거트"]

#### 파일 저장 및 불러오기

In [8]:
# with open(os.path.join(data_path, "menu_unique.txt"), "w") as file:
#     file.write('\n'.join(menus))

In [9]:
menu_unique = []

with open(os.path.join(data_path, "menu_unique.txt"), 'r', encoding='utf-8') as f:
    for line in f:
        for sc in STOPWORDS_CRAWLED:
            line = line.replace("\n","").replace(sc[0],sc[1])
        menu_unique.append(line)

len(menu_unique)

165064

# 식품영양성분 DB 데이터 불러오기 및 분류

In [10]:
# zip_path = os.path.join(data_path, "Integrated Food Nutrients DB_Food.zip")
# with zipfile.ZipFile(zip_path, "r") as target_file:
#     for member in target_file.infolist():
#         # Decode the filename using the correct encoding
#         filename = member.filename.encode('cp437').decode('cp949')
#         target_file.extract(member, path=data_path)

In [11]:
db=pd.read_excel('content/통합 식품영양성분DB_음식.xlsx',sheet_name="Sheet1")
text=[]
for i in db["식품명"].values.tolist() :
    text.append(i)
    with open(os.path.join(data_path, "database_menu.txt"), 'w', encoding="utf-8") as f:
        f.write("\n".join(text))

In [12]:
# 식품영양성분DB 메뉴 목록
db_menus = []
# 식품영양성분DB 중복 된 메뉴 목록(선택 필요)
db_bracket_menus = []

with open(r'content\database_menu.txt', 'r', encoding="utf-8") as f:
    for line in f:
        for sd in STOPWORDS_DB:
            line = line.strip().replace(' ', '').replace(sd[0],sd[1])
        if "(" in line:
            db_bracket_menus.append(line)
        else:
            db_menus.append(line)

with open(os.path.join(data_path, "db_menus.txt"), "w") as f:
    f.write("\n".join(db_menus))
with open(os.path.join(data_path, "db_bracket_menus.txt"), "w") as f:
    f.write("\n".join(db_bracket_menus))

In [13]:
db_menus = []

with open(os.path.join(data_path, "db_menus.txt"), 'r') as f:
    for line in f:
        db_menus.append(line.replace("\n",""))

len(db_menus)

703

# 급식데이터와 식품영양성분DB 데이터 매칭

## 식품영양성분DB 데이터 정렬

- 메뉴명의 길이 기준으로 내림차순 정렬
    - ex) 참치김치찌개 > 김치찌개 > 김치

In [14]:
sorted_db_menus = sorted(db_menus, key=len)[::-1]
sorted_db_menus

['돼지고기메추리알장조림',
 '돼지고기메추리알장조림',
 '두부쇠고기고추장찌개',
 '쇠고기메추리알장조림',
 '소고기메추리알장조림',
 '참치마요네즈삼각김밥',
 '고추장불고기삼각김밥',
 '간장양념닭다리튀김',
 '두부애호박된장찌개',
 '호박오가리나물무침',
 '잔멸치꽈리고추볶음',
 '콩나물고춧가루무침',
 '고구마줄기나물무침',
 '닭고기대파꼬치구이',
 '간장양념닭다리구이',
 '고추장마늘쫑장아찌',
 '돼지고기김치찌개',
 '두릅초고추장무침',
 '닭가슴살샌드위치',
 '잔멸치풋고추볶음',
 '잔멸치견과류볶음',
 '쇠고기고추장볶음',
 '마늘쫑잔멸치볶음',
 '돼지고기채소볶음',
 '돼지고기김치볶음',
 '닭가슴살피망볶음',
 '꽈리고추멸치볶음',
 '참치마요삼각김밥',
 '열무된장나물무침',
 '간장마늘쫑장아찌',
 '숯불갈비삼각김밥',
 '호박고추장찌개',
 '콩나물된장찌개',
 '순두부김치찌개',
 '쇠고기된장찌개',
 '쇠고기샤브샤브',
 '두부고추장찌개',
 '메추리알장조림',
 '돼지고기장조림',
 '스크램블드에그',
 '돼지고기완자전',
 '일본매실장아찌',
 '풋고추된장무침',
 '오징어채소무침',
 '새싹채소샐러드',
 '도라지오이생채',
 '닭가슴살샐러드',
 '느타리버섯무침',
 '골뱅이국수무침',
 '햄치즈샌드위치',
 '콤비네이션피자',
 'BLT샌드위치',
 '멸치풋고추볶음',
 '죽순쇠고기볶음',
 '잔멸치고추볶음',
 '쇠고기채소볶음',
 '쇠고기버섯볶음',
 '새송이버섯볶음',
 '문어고추장볶음',
 '멸치마늘쫑볶음',
 '멸치견과류볶음',
 '느타리버섯볶음',
 '표고버섯볶음밥',
 '토마토스파게티',
 '토란대나물무침',
 '취나물된장무침',
 '씀바귀나물무침',
 '시래기나물무침',
 '시금치나물무침',
 '머위대나물무침',
 '도라지나물무침',
 '고춧잎나물무침',
 '고사리나물무침',
 '얼갈이배추김치',
 '무시래기된장국',
 '새송이버섯구이',
 '새우젓두부찌개',
 '양송이버섯볶음',
 '멸치풋고추볶

## 매칭

In [15]:
# 식품영양성분DB 메뉴 중 매칭 된 메뉴
menus = dict()

# 식품영양성분DB 메뉴 중 매칭이 되지 않은 메뉴
not_included_menus = dict()

# 논리 에러 처리 문제 발생
# 반복문 처리를 위한 기준
menu_unique_cnt = len(menu_unique)
change_menu_unique_cnt = 0

print(menu_unique_cnt)

# 매칭 전 데이터와 매칭 후 데이터의 개수가 같을 때까지 반복 실행
while menu_unique_cnt-change_menu_unique_cnt != 0:  
    menu_unique_cnt = len(menu_unique)
    for db_menu in tqdm(sorted_db_menus):
        menu_list = []
        for menu_idx, menu in enumerate(menu_unique):
            #식품영양성분DB 메뉴명과 급식데이터의 메뉴명의 끝부분과 일치할 경우 매칭
            #ex) 김치찌개 => 참치김치찌개 => 매칭
            #    김치찌개 => 김치찌개소스 => 비매칭
            start_idx = len(menu)-len(db_menu)
            if start_idx >=0 and db_menu == menu[start_idx:]:
                menu_list.append(menu)
                #매칭 된 급식데이터 제거
                menu_unique[menu_idx]=""
        
        #논리 에러 처리 문제 발생
        #반복문 처리를 하기 위한 작업
        #반복 시, 이미 이전에 매칭이 완료 된 리스트의 경우, 이번 사이클에선 매칭이 되지 않았다고 판별함.
        #따라서 이전에 매칭이 완료 된 리스트라는 것을 알려줘야함.
        check_not_included_menus = []
        check_not_included_menus.extend(list(menus.keys()))
        check_not_included_menus.extend(list(not_included_menus.keys()))
        
        if len(menu_list) > 0:
            #논리 에러 처리 문제 발생
            #반복문 처리를 하기 위한 작업
            #이전에 매칭이 되어 있던 메뉴의 경우, extend
            #아닐 경우, append
            if db_menu in list(menus.keys()):
                menus[db_menu].extend(menu_list)
            else: menus[db_menu] = menu_list
        elif not db_menu in check_not_included_menus:
            not_included_menus[db_menu] = menu_list
            
    menu_unique = list(set(menu_unique))
    change_menu_unique_cnt = len(menu_unique)
    print(len(not_included_menus))
    print(menu_unique_cnt-change_menu_unique_cnt)

165064


100%|███████████████████████████████████████████████████████████████| 703/703 [00:41<00:00, 17.06it/s]


76
37192


100%|███████████████████████████████████████████████████████████████| 703/703 [00:38<00:00, 18.05it/s]

76
0


In [16]:
#식품영양성분DB 메뉴 중 학교급식정보 메뉴에서 검출 된 목록
with open(os.path.join(data_path, "include_db_menus.txt"), "w", encoding="euc-kr") as f:
    for key, value in menus.items():
        f.write(f"{key}: {value}\n")
        
#식품영양성분DB 메뉴 중 학교급식정보 메뉴에서 검출 되지 않은 목록
with open(os.path.join(data_path, "not_include_db_menus.txt"), "w", encoding="euc-kr") as f:
    for key, value in not_included_menus.items():
        f.write(f"{key}: {value}\n")

#식품영양성분DB 메뉴와 매칭 되지 않은 학교급식정보 메뉴 목록
with open(os.path.join(data_path, "remain_unique_menus.txt"), "w", encoding="euc-kr") as f:
    f.write("\n".join(menu_unique))

# 분할 된 매칭 데이터 병합 및 중복 제거

In [17]:
menus = dict()

with open(os.path.join(data_path, "include_db_menus.txt"), "r", encoding="euc-kr") as f:
    for line in f:
        key, value = line.split(":")
        menus[key] = eval(value)

menus["돼지고기메추리알장조림"]

['깍둑돼지고기메추리알장조림', '돼지고기메추리알장조림']

In [18]:
with open(os.path.join(data_path, "normalizated_cafeteria_dish[defalut].txt"), "r", encoding="utf-8") as f:
    for line in f:
        key, value = line.split(":")
        if len(eval(value)) == 0: continue
        if key in menus.keys():
            menu_list = menus[key]
            menu_list.extend(eval(value))
            menu_list = list(set(menu_list))
            menus[key] = menu_list
        else:
            menus[key] = eval(value)

In [19]:
str_menu[19560:19570]

NameError: name 'str_menu' is not defined

In [ ]:
# with open(os.path.join(data_path, "changed[121-240].txt"), "r", encoding="utf-8") as f:
#     str_menu = f.read()
#     json_menu = json.loads(str_menu)
    
# for key, value in json_menu.items():
#     if len(value) == 0: continue
#     if key in menus.keys():
#         menu_list = menus[key]
#         menu_list.extend(value)
#         menu_list = list(set(menu_list))
#         menus[key] = menu_list
#     else:
#         menus[key] = eval(value)

In [ ]:
# with open(os.path.join(data_path, "241_360.txt"), "r", encoding="utf-8") as f:
#     for line in f:
#         key, value = line.split(":")
#         if len(eval(value)) == 0: continue
#         if key in menus.keys():
#             menu_list = menus[key]
#             menu_list.extend(eval(value))
#             menu_list = list(set(menu_list))
#             menus[key] = menu_list
#         else:
#             menus[key] = eval(value)

In [ ]:
# with open(os.path.join(data_path, "361_480.txt"), "r", encoding="utf-8") as f:
#     for line in f:
#         key, value = line.split(":")
# #         print(key)
#         if len(eval(value)) == 0: continue
#         if key in menus.keys():
#             menu_list = menus[key]
#             menu_list.extend(eval(value))
#             menu_list = list(set(menu_list))
#             menus[key] = menu_list
#         else:
#             menus[key] = eval(value)

In [ ]:
# with open(os.path.join(data_path, "481-600.txt"), "r", encoding="utf-8") as f:
#     str_menu = f.read()
#     json_menu = json.loads(str_menu)
    
# for key, value in json_menu.items():
#     if len(value) == 0: continue
#     if key in menus.keys():
#         menu_list = menus[key]
#         menu_list.extend(value)
#         menu_list = list(set(menu_list))
#         menus[key] = menu_list
#     else:
#         menus[key] = eval(value)

# 급식데이터 수정

In [20]:
key_value_change = dict()

for key, value in tqdm(menus.items()):
    if key == None: continue
    if value == None: continue
    for v in value:
        if v in key_value_change.keys():
            try:
                key_list = key_value_change[v]
                key_list.append(key)
                key_list = list(set(key_list))
                key_value_change[v] = key_list
            except:
#                 print("들어오니?", v, key,key_value_change[v])
                pass
#             print(v, key_value_change[v])
        else:
            key_value_change[v] = [key]
    
# key_value_change

100%|████████████████████████████████████████████████████████████| 729/729 [00:00<00:00, 26986.23it/s]


In [21]:
print(key_value_change["고구마전"])

['고구마전']


In [22]:
sorted_data = sorted(key_value_change.items(), key=lambda x: len(x[1]), reverse=True)

dup_menu_list = dict()
# 결과 출력
cnt = 0
for key, value in sorted_data:
    cnt += 1
    if len(value) <= 1: break
#     print(key, ":", value)
    dup_menu_list[key] = value
    
print(cnt)

#중복항목
with open(os.path.join(data_path, "dup_menu_list.txt"), "w", encoding="euc-kr") as f:
    for key, value in dup_menu_list.items():
        f.write(f"{key}: {value}\n")

272


In [23]:
str_menus = []

# 한글자 메뉴 제외하고 메뉴 리스트 생성
for menu_idx, menu in tqdm(enumerate(menu_csv.menu)):
    matches = re.findall("[가-힣]+", menu)
    for match_idx, match in enumerate(matches):
        for sc in STOPWORDS_CRAWLED:
            match = match.replace(sc[0],sc[1])
        if match in key_value_change.keys():
            if len(key_value_change[match]) == 0:
                print(match)
                matches[match_idx] = ""
            matches[match_idx] = key_value_change[match][0]
        else:
            matches[match_idx] = ""
    matches = list(set(matches))
    if "" in matches:
        matches.remove("")
    str_menus.append(" ".join(matches))
    
menu_csv["menu2"] = str_menus

1912349it [00:16, 113365.33it/s]


In [24]:
menu_csv.to_csv("content/test_csv.csv", encoding="euc-kr", index=False)

In [25]:
menu_csv[-5:]

,sch_nm,menu_date,dt,menu,menu2
572278,흥덕고등학교 (충청북도교육청),2019.12.26,중식,혼합잡곡밥5. 두부동태찌개5.8.9. 호두멸치볶음5.6. 비엔나소시지그라탕-문어모양...,잡곡밥 배추김치 멸치볶음 동태찌개
572279,흥덕고등학교 (충청북도교육청),2019.12.27,석식,백미밥 청경채된장국5.6.8.9. 무생채13. 파절이양배추무침 삼겹살오븐구이10. ...,무생채 배추김치
572280,흥덕고등학교 (충청북도교육청),2019.12.27,중식,콩나물밥/부추양념간장5.6. 유부장국5.6.8.9. 열무김치 왕새우크레페(해우푸드)...,콩나물밥 열무김치
572281,흥덕고등학교 (충청북도교육청),2019.12.30,석식,백미밥 쥬키니된장국5.6.8.9. 부추야채무침 목살큐브버섯오븐구이5.6.10.13....,배추김치
572282,흥덕고등학교 (충청북도교육청),2019.12.30,중식,카레라이스2.5.6.10. 두부김치국5.8.9. 소떡소떡오븐구이(100g)5.6.1...,깍두기 김치국 카레라이스


In [26]:
for menu2 in menu_csv.menu2:
    if "스파게티" in menu2:
        print(menu2)

크림스파게티
오일스파게티 도넛
스테이크 볶음밥 배추김치 크림소스스파게티
배추김치 볶음밥 토마토스파게티 모닝빵
배추김치 토마토스파게티
배추김치 토마토스파게티 햄버거
배추김치 토마토스파게티
콤비네이션피자 배추김치 토마토스파게티
배추김치 토마토스파게티
배추김치 토마토스파게티
배추김치 토마토스파게티
배추김치 토마토스파게티
배추김치 토마토스파게티
배추김치 토마토스파게티
배추김치 찹쌀떡 토마토스파게티
배추김치 토마토스파게티
배추김치 토마토스파게티
배추김치 토마토스파게티
배추김치 토마토스파게티
배추김치 토마토스파게티 양상추샐러드
배추김치 마늘빵 토마토스파게티 양상추샐러드
순두부찌개 배추김치 토마토스파게티 현미밥
배추김치 크림스파게티
스테이크 나박김치 크림소스스파게티 모닝빵
스테이크 배추김치 마늘빵 토마토스파게티
스테이크 배추김치 마늘빵 토마토스파게티
미트볼토마토스파게티 배추김치 감자튀김
마늘빵 토마토스파게티
마늘빵 토마토스파게티
토마토스파게티
깍두기 크림스파게티
함박스테이크 크림스파게티 마늘빵
깍두기 콩나물국 김치볶음밥 마늘빵 토마토스파게티
돈까스 크림스파게티 배추김치 양상추샐러드
함박스테이크 크림스파게티 배추김치 양배추샐러드
크림스파게티 양송이스프
배추김치 토마토소스스파게티
함박스테이크 잡곡밥 배추김치 김치콩나물국 토마토소스스파게티
배추김치 토마토소스스파게티 마늘빵
토마토소스스파게티
토마토소스스파게티
깍두기 함박스테이크 양배추샐러드 토마토소스스파게티
토마토소스스파게티 마늘빵
배추김치 토마토소스스파게티 양상추샐러드
깍두기 함박스테이크 배추김치 토마토소스스파게티
배추김치 토마토소스스파게티 마늘빵
배추김치 수수밥 미소된장국 토마토소스스파게티
토마토소스스파게티 감자튀김
배추김치 토마토소스스파게티
배추김치 토마토소스스파게티
깍두기 감자샐러드 토마토소스스파게티
배추김치 토마토소스스파게티 양상추샐러드
토마토소스스파게티 감자튀김
배추김치 토마토소스스파게티 감자튀김
돈까스 김치콩나물국 크림소스스파게티 오이지무침
양념치킨 배추김치 크림소스스파게티 미소된장국
크림소스스파게티 배추김치 단무지

배추김치 등심돈까스 토마토스파게티
순두부찌개 크림스파게티 배추김치 차조밥
순두부찌개 잡곡밥 크림스파게티 배추김치
토마토스파게티
잡곡밥 크림스파게티 버섯찌개 배추김치
크림스파게티 스테이크 배추김치 양상추샐러드
배추김치 크림스파게티 무조림
배추김치 크림스파게티
크림스파게티 김치콩나물국 파김치
함박스테이크 크림스파게티 오이소박이 배추김치
오징어무침 크림스파게티 콩나물국 배추김치
미역줄기볶음 함박스테이크 크림스파게티 콩나물국 배추김치
배추김치 함박스테이크 크림스파게티
크림스파게티 스테이크 배추김치 양상추샐러드
배추김치 크림스파게티 무조림
배추김치 크림스파게티
크림스파게티 김치콩나물국 파김치
함박스테이크 크림스파게티 오이소박이 배추김치
오징어무침 크림스파게티 콩나물국 배추김치
미역줄기볶음 함박스테이크 크림스파게티 콩나물국 배추김치
배추김치 함박스테이크 크림스파게티
크림스파게티 스테이크 배추김치 양상추샐러드
배추김치 크림스파게티 무조림
배추김치 크림스파게티
크림스파게티 김치콩나물국 파김치
함박스테이크 크림스파게티 오이소박이 배추김치
오징어무침 크림스파게티 콩나물국 배추김치
미역줄기볶음 함박스테이크 크림스파게티 콩나물국 배추김치
배추김치 함박스테이크 크림스파게티
스테이크 크림소스스파게티
함박스테이크 오일스파게티
함박스테이크 오일스파게티 감자튀김
크림소스스파게티
함박스테이크 오일스파게티
크림소스스파게티 콩나물국
크림소스스파게티 함박스테이크 감자튀김
함박스테이크 크림스파게티 감자튀김
감자튀김 크림스파게티 스테이크 볶음밥 떡볶이 순대 양상추샐러드
함박스테이크 오일스파게티
크림소스스파게티 함박스테이크 감자튀김
감자튀김 스테이크 파김치 크림소스스파게티
함박스테이크 오일스파게티
크림스파게티 탕수육 스테이크 볶음밥 떡볶이 순대 양상추샐러드
단호박샐러드 크림소스스파게티
찐빵 크림소스스파게티
스테이크 크림소스스파게티
함박스테이크 오일스파게티
불고기피자 크림스파게티
스테이크 크림소스스파게티
함박스테이크 오일스파게티
함박스테이크 오일스파게티 감자튀김
크림소스스파게티
함박스테이크 오일스파게티
